In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from glob import glob
from skimage.io import imread
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
all_tif_images=glob('../input/BBBC010_v1_images/*_w1_*.tif')
all_fg_images=glob('../input/BBBC010_v1_foreground/*.png')
image_df=pd.DataFrame([{'gfp_path': f} for f in all_tif_images])
def _get_light_path(in_path):
    w2_path='_w2_'.join(in_path.split('_w1_'))
    glob_str='_'.join(w2_path.split('_')[:-1]+['*.tif'])
    m_files=glob(glob_str)
    if len(m_files)>0:
        return m_files[0]
    else:
        return None
image_df['light_path']=image_df['gfp_path'].map(_get_light_path)
image_df=image_df.dropna()
image_df['base_name']=image_df['gfp_path'].map(lambda x: os.path.splitext(os.path.basename(x))[0])
# clearly this is not the case
# <plate>_<wellrow>_<wellcolumn>_<wavelength>_<fileid>.tif
# Columns 1-12 are positive controls treated with ampicillin. Columns 13-24 are untreated negative controls.
# we apply a new rule
# 1649_1109_0003_Amp5-1_B_20070424_A01_w1_9E84F49F-1B25-4E7E-8040-D1BB2D7E73EA.tif
# junk_junk_junk_junk_junk_date_RowCol_wavelength_id.tif

image_df['plate_rc']=image_df['base_name'].map(lambda x: x.split('_')[6])
image_df['row']=image_df['plate_rc'].map(lambda x: x[0:1])
image_df['column']=image_df['plate_rc'].map(lambda x: int(x[1:]))
image_df['treated']=image_df['column'].map(lambda x: x<13)
image_df['wavelength']=image_df['base_name'].map(lambda x: x.split('_')[7])

image_df['mask_path']=image_df['plate_rc'].map(lambda x: '../input/BBBC010_v1_foreground/{}_binary.png'.format(x))
print('Loaded',image_df.shape[0],'datasets')
image_df.sample(3)

In [ ]:
%matplotlib inline
test_image_row=list(image_df.query('treated').sample(1).T.to_dict().values())[0]
test_img=imread(test_image_row['light_path'])
test_gfp=imread(test_image_row['gfp_path'])
test_bg=imread(test_image_row['mask_path'])
print('Test Image:',test_img.shape)

fig, ((ax_light,ax_gfp, ax4),(ax2 ,ax3, _)) = plt.subplots(2,3, figsize = (10,6))
ax_light.imshow(test_img,cmap='gray')
ax_light.set_title('Light-field Image'.format(**test_image_row))

ax_gfp.imshow(np.sqrt(test_gfp),cmap='BuGn')
ax_gfp.set_title('GFP Image'.format(**test_image_row))

ax2.hist(test_img.ravel())
ax2.set_title('Light Distribution')

ax3.hist(test_gfp.ravel())
ax3.set_title('GFP Distribution')

ax4.imshow(test_bg, cmap = 'bone')
ax4.set_title('Segmented')

In [ ]:
%matplotlib inline
test_image_row=list(image_df.query('treated==False').sample(1).T.to_dict().values())[0]
test_img=imread(test_image_row['light_path'])
test_gfp=imread(test_image_row['gfp_path'])
test_bg=imread(test_image_row['mask_path'])
print('Test Image:',test_img.shape)

fig, ((ax_light,ax_gfp, ax4),(ax2 ,ax3, _)) = plt.subplots(2,3, figsize = (10,6))
ax_light.imshow(test_img,cmap='gray')
ax_light.set_title('Light-field Image'.format(**test_image_row))

ax_gfp.imshow(np.sqrt(test_gfp),cmap='BuGn')
ax_gfp.set_title('GFP Image'.format(**test_image_row))

ax2.hist(test_img.ravel())
ax2.set_title('Light Distribution')

ax3.hist(test_gfp.ravel())
ax3.set_title('GFP Distribution')

ax4.imshow(test_bg, cmap = 'bone')
ax4.set_title('Segmented')

Load the data on individual worms

In [ ]:
worm_df=pd.read_csv('../input/BBBC010_v1_foreground_eachworm.csv')
worm_df.sample(3)

Summarize the count and area

In [ ]:
worm_summary_df=worm_df.groupby('plate_rc').agg({'worm_id':'max', 'worm_pixel_area': 'sum'}).reset_index().rename(columns={'worm_id':'worm_count'})
worm_summary_df.sample(3)

In [ ]:
# The npz was saved directly from a pandas df and so it is a bit uglier (hence the additional ravel), h5 was unfortunately too poorly compressed
with np.load('../input/BBBC010_v1_foreground_eachworm.npz') as mask_npz:
    mask_df=pd.DataFrame({k:[iv for ik,iv in v.ravel()[0].items()] 
                          for k,v in mask_npz.items() if 'path' not in k})
mask_df.sample(2)

# Simple Analysis of Image Intensity and Std

In [ ]:
image_df['treated']=image_df['treated'].map(lambda x: 'ampicillin' if x else 'negative control')
image_df.sample(3)

In [ ]:
%%time
image_df['light_mean']=image_df['light_path'].map(lambda x: np.mean(imread(x)))
image_df['gfp_mean']=image_df['gfp_path'].map(lambda x: np.mean(imread(x)))
image_df['light_sd']=image_df['light_path'].map(lambda x: np.std(imread(x)))
image_df['gfp_sd']=image_df['gfp_path'].map(lambda x: np.std(imread(x)))
image_df.sample(3)

# Showing Correlations
Here we can see correlations for the dataset and it is very evident that the GFP signal quite strongly separates living from dead worms

In [ ]:
sns.pairplot(image_df.drop(['column'],1),hue='treated')

# Adding the worm information
Now we can what additional information we get from segmenting the worms out

In [ ]:
full_df=image_df.merge(worm_summary_df,on='plate_rc')

In [ ]:
sns.pairplot(full_df.drop(['column'],1),hue='treated')